## Step 8 - Deploy the application on the Production Centre

Step 8 of the tutorial uses a Jupyter Notebook to create a Web Processing Request to deploy the data transformation application on the Production Centre. 

* First do the imports of the Python libraries required

In [1]:
import jenkins
import time
import lxml.etree as etree
import sys
import requests
import os
import string
import hashlib
import urllib2
import pytz
from datetime import datetime
import time
import owslib
from owslib.wps import monitorExecution
import uuid
from owslib.wps import WebProcessingService
import getpass 

* Create a simple Python class to read the elements of the pom with a constructor taking as parameter the path to the pom file and methods to retrieve the version, artifact id and community. This information is used to create the build request

In [17]:
api_key = getpass.getpass('And your Ellip API key:')

And your Ellip API key:········


In [13]:
app = dict([('artifact_id', 'wfp-01-01-01'),
            ('version', '0.4'),
            ('repository', 'Github Organizations'),
            ('community', 'ec-better')])

* Connect to the WPS server and do a GetCapabilities request to list the available process:

In [6]:
wps_url = 'https://ec-better-apps-deployer.terradue.com/zoo-bin/zoo_loader.cgi'

wps = WebProcessingService(wps_url, verbose=False, skip_caps=True)

wps.getcapabilities()

In [7]:
for index, elem in enumerate(wps.processes):
    print(index, elem.identifier)

(0, 'TerradueUnDeployProcess')
(1, 'ec_better_wfp_01_01_01_wfp_01_01_01_0_1')
(2, 'GetStatus')
(3, 'TerradueDeployProcess')


* Select the **DeployProcess** process to submit a DescribeProcess request and list the inputs:

In [21]:
process_id = 'TerradueDeployProcess'
        
process = wps.describeprocess(process_id)

for input in process.dataInputs:
    print(input.identifier)
    
    
    
for output in process.processOutputs:
    print(output.identifier)

applicationPackage
apikey
deployResult


* Build the Python dictionary with the inputs:

In [24]:
ows_context_url = '/%s/_applications/%s/%s/%s/%s-%s-application-context.xml' % (app['community'], 
                                                                   app['community'], 
                                                                   app['artifact_id'],
                                                                   app['version'], 
                                                                   app['artifact_id'],
                                                                    app['version'])
#/ec-better/_applications/ec-better/wfp-01-01-01/0.4/wfp-01-01-01-0.4-application-context.xml    

In [25]:
ows_context_url

'/ec-better/_applications/ec-better/wfp-01-01-01/0.4/wfp-01-01-01-0.4-application-context.xml'

In [26]:
inputs = [('applicationPackage', ows_context_url),
          ('apikey', api_key)]

* Submit the Execute WPS request:

In [27]:
execution = owslib.wps.WPSExecution(url=wps.url)

execution_request = execution.buildRequest(process_id, 
                                           inputs, 
                                           output = [('deployResult', False)])
        
execution_response = execution.submitRequest(etree.tostring(execution_request))
        
execution.parseResponse(execution_response)

* Monitor the request:

In [28]:
monitorExecution(execution)

* Check if the application was successfully deployed

In [29]:
if execution.isSucceded(): 
            
    print 'Application deployed!'
            
else: 

    print 'Application not deployed :-('

Application deployed!


* List the WPS process available, the newly deployed process must be amongst the exposed process

In [34]:
wps = WebProcessingService(wps_url, verbose=False, skip_caps=True)

wps.getcapabilities()

In [35]:
for index, elem in enumerate(wps.processes):
    print(index, elem.identifier)

(0, 'ec_better_wfp_01_01_01_wfp_01_01_01_0_4')
(1, 'TerradueUnDeployProcess')
(2, 'ec_better_wfp_01_01_01_wfp_01_01_01_0_1')
(3, 'GetStatus')
(4, 'TerradueDeployProcess')
